In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier

: 

In [ ]:
CC1 = pd.read_csv(f"D:\UNNC\Y3\Credit-Risk-Analyzing\MLP\useful_data\CCdata.csv", low_memory=False)

In [3]:
def process_ccdata(CC1, dels_threshold = 7):

  CC2 = CC1.copy()
  CC2.rename(columns={'STATEMENT_NUMBER': 'acc_age', 'HIGHEST_BUCKET': 'dels'}, inplace=True)

  # only include application channel as special case
  CC2['appInternet'] = (CC2['APPLICATION_CHANNEL']=='Internet').astype(int)
    
  CC2 = CC2[['id', 'dels', 'acc_age', 'age', 'maa_log', 'appInternet', 'EMPLOY_STATUS']]

  # get indicator variables for employment status
  CC2 = pd.get_dummies(CC2, prefix=['emp'], columns=['EMPLOY_STATUS'], 
                       drop_first=True, dummy_na=True, dtype=int)

  # remove missing data
  n = CC2.shape[0]
  CC2.dropna(inplace=True)
  print(f"Number of rows with missing values removed = {n-CC2.shape[0]}.")
    
  # set a threshold on dels values
  CC2['dels'] = CC2['dels'].where(CC2['dels']<=dels_threshold, dels_threshold)

  # set up previous state as a feature
  CC2['dels_prev'] = CC2.groupby('id')['dels'].shift(1)
    
  # remove rows where no previous delinquency (normally first)
  ix = CC2.index[CC2['dels_prev'].isnull()].tolist()
  CC2.drop(ix, inplace=True)

  # show some statistics
  n = CC2.shape[0]
  nid = CC2['id'].nunique()
  print(f"Number of rows = {n}")
  print(f"Number of accounts = {nid}")
  print(f"Months per account (mean) = {n/nid}")

  print("Number in delinquency state:")
  for i in range(dels_threshold+1):
    ndels = CC2['dels'].value_counts().get(i, 0)
    print(f"  {i} is {ndels} ({100*ndels/n} %)")

  return CC2

# Split into training and test
def train_test_split_cc(CC, test_size=0.2, random_state=42):
  u_id=CC['id'].unique()
  sa, sb = train_test_split(u_id, test_size=test_size, random_state=random_state)
  
  # sort according to the Loan Sequence Number 
  CC_a = CC[CC["id"].isin(sa)].reset_index(drop=True)
  CC_b = CC[CC["id"].isin(sb)].reset_index(drop=True)

  # ensure the data is sorted in correct order
  CC_a = CC_a.sort_values(by=["id", "acc_age"])
  CC_b = CC_b.sort_values(by=["id", "acc_age"])

  return (CC_a, CC_b)

def transition_matrix(M):
  return pd.crosstab(M['dels_prev'], M['dels'], normalize='index').to_numpy()

def predict_from_tm(TM, X, horizon=1):
  return np.matmul(X, np.linalg.matrix_power(TM, horizon))

def roll_rate(YP):
  S = np.sum(YP, axis=0)
  return S/np.sum(S)
def rmse(V1,V2):
  return np.sqrt(np.sum((V1-V2)**2))

# compute Brier scores and entropy
# preds = prediction probability
# Y = outcome states
def evaluate(preds, Y):
    
  D = np.square(preds - Y)

  # replace 0's with small threshold value to allow for use of log
  t = np.exp(-10)
  preds2 = np.where(preds>t, preds, t)
  E = -np.log(preds2)
  E = np.multiply(E, Y)

  Bcol = np.mean(D, axis=0)
  Erowsum = np.sum(E, axis=1)

  # returns in tuple:
  #  [0] Brier score by delinquency state;
  #  [1] Mean Brier score;
  #  [2] Entropy (mean) by delinquency;
  #  [3] Overall entropy (mean)
  #  [4] RMSE of roll rate predictions;
  #  [5] size of prediction set
  return (Bcol, np.mean(Bcol), 
          np.divide(np.sum(E, axis=0), 
          np.sum(Y, axis=0)), 
          np.mean(Erowsum),
          rmse(roll_rate(Y), roll_rate(preds) ),
          preds2.shape[0])

#print(evaluate2(preds1, Y_test, 2, FM_b))

def print_evaluation(eval, header_text="Evaluation:"): 
  print(header_text)
  print(f"  Test set observations = {eval[5]};")
  print(f"  Brier score = {eval[1]};")
  print(f"    by delinquency: {eval[0]};")
  print(f"  Entropy (mean) = {eval[3]};")
  print(f"    by delinquency: {eval[2]};")
  print(f"  Roll rate RMSE: {eval[4]};")

def shift_dels(CC, horizon=1):
  if  horizon<=1:
    CC2 = CC
  else:
    CC2 = CC.copy()
    CC2['dels'] = CC2.groupby('id')['dels'].shift(1-horizon)
      
    # Remove rows with no target outcome
    ix = CC2.index[CC2['dels'].isnull()].tolist()
    CC2.drop(index=ix, inplace=True)
      
  return CC2

def encode_dels(CCin, include=None, horizon=1):
  CC = shift_dels(CCin, horizon)
    
  # Encode the previous delinquency flag to construct X features
  encoder_dels_prev = OneHotEncoder(sparse_output=False)  # Use sparse=False to get a dense NumPy array as output.
  # Fit the encoder on the training data and transform both training and test data
  encoder_dels_prev.fit(CCin[['dels_prev']])
  # Encode the next delinquency flag to construct Y target
  encoder_dels = OneHotEncoder(sparse_output=False)  # Use sparse=False to get a dense NumPy array as output.
  # Fit the encoder on the training data and transform both training and test data
  encoder_dels.fit(CCin[['dels']])

  X_status = encoder_dels_prev.transform(CC[['dels_prev']])
  if  include is None:
    X = X_status
  else:
 #   X = np.hstack(([X_status, CC[include].values.reshape(-1,1)]))
    X = np.hstack(([X_status, CC[include].to_numpy()]))
      
  Y = encoder_dels.transform(CC[['dels']])
  return X,Y

# One level of propagation
def propagate_mlp_mc_once(mlp_in, X_test, Pin=None):
  if  Pin is None:
    Paggr = mlp_in.predict_proba(X_test)

  else:
    X = X_test.copy()
    for state in range(n_states+1):
    
      # change all dels_prev to state
      for i in range(n_states+1):
        if  i==state:
          X[:,i] = 1
        else:
          X[:,i] = 0
        
      # get new prediction and integrate
      Pnext = np.multiply(Pin[:,state:state+1], mlp_in.predict_proba(X))
      if  state==0:
        Paggr = Pnext
      else:
        Paggr = Paggr + Pnext
          
  return Paggr

def propagate_mlp_mc(mlp_in, X_test, horizon=1):
  P = None
  for i in range(0,horizon):
    P = propagate_mlp_mc_once(mlp_in, X_test, Pin=P)
  return P

In [4]:
print(CC1.columns)
print(CC1['EMPLOY_STATUS'].value_counts())
print(CC1['EMPLOY_STATUS'].isnull().sum())
print(CC1['APPLICATION_CHANNEL'].value_counts())
print(CC1['APPLICATION_CHANNEL'].isnull().sum())
print(CC1['age'].describe())
print(CC1['age'].isnull().sum())
print(CC1['maa_log'].describe())
print(CC1['maa_log'].isnull().sum)

Index(['id', 'rel_mth', 'age', 'EMPLOY_STATUS', 'MONTHS_AT_ADDRESS',
       'APPLICATION_CHANNEL', 'STATEMENT_NUMBER', 'TOTAL_BAL', 'CREDIT_LIMIT',
       'CASH_BAL', 'STMT_YEAR_MONTH', 'ACCOUNT_STATUS', 'HIGHEST_BUCKET',
       'PAYMENTS_AMT', 'PAYMENTS_NO', 'stmt_year', 'stmt_month', 'CL',
       'TOTAL_BAL_PREV', 'interest_BAL', 'interest_BAL_cat', 'active', 'def',
       't_fe', 't', 't1or2', 't_2', 't_log', 't_log_2', 'c1m', 'c3m',
       'age_unknown', 'age18_30', 'age30_40', 'age40_50', 'age50', 'emp',
       'tenure', 'app', 'maa', 'maa_log', 'maa_missing', 'randn'],
      dtype='object')
EMPLOY_STATUS
EM    1012247
SE     163027
RE      65312
HO      44097
ST      24249
Name: count, dtype: int64
46212
APPLICATION_CHANNEL
Mail          585978
Internet      465970
Cold Calls     95186
Inserts        92403
Other          79689
Doordrops      35918
Name: count, dtype: int64
0
count    1.353739e+06
mean     3.915811e+01
std      1.225891e+01
min      1.800000e+01
25%      3.000000e

In [5]:
n_states=7
CC_a, CC_b = train_test_split_cc(process_ccdata(CC1, n_states), random_state=43)

TM = transition_matrix(CC_a)
np.savetxt("CC_mc_tm.csv", TM, delimiter=",")

X_train, Y_train = encode_dels(CC_a)
X_test, Y_test = encode_dels(CC_b)
preds1 = predict_from_tm(TM, X_test)
eval1 = evaluate(preds1, Y_test)
print_evaluation(eval1, "Horizon=1; Evaluation of Transition matrix:")

# Try Identity matrix as baseline
preds2 = predict_from_tm(np.identity(TM.shape[0]), X_test)
eval2 = evaluate(preds2, Y_test)
print_evaluation(eval2, "Horizon=1; Evaluation of Identity matrix:")

Number of rows with missing values removed = 1405.
Number of rows = 1313959
Number of accounts = 39601
Months per account (mean) = 33.17994495088508
Number in delinquency state:
  0 is 1102807 (83.93009218704694 %)
  1 is 45057 (3.429102430136709 %)
  2 is 15881 (1.2086374080165363 %)
  3 is 10870 (0.8272708661381367 %)
  4 is 9353 (0.7118182530809561 %)
  5 is 8726 (0.6640998691740001 %)
  6 is 7818 (0.5949957342656811 %)
  7 is 113447 (8.633983252141048 %)
Horizon=1; Evaluation of Transition matrix:
  Test set observations = 261237;
  Brier score = 0.022616135328070365;
    by delinquency: [0.06835135 0.03275603 0.01180075 0.0081764  0.00707605 0.00656163
 0.0059511  0.04025577];
  Entropy (mean) = 0.4543986897348745;
    by delinquency: [0.10673306 3.17961362 3.91183293 4.12500132 4.16598404 4.11463924
 4.14587065 1.0378885 ];
  Roll rate RMSE: 0.0010476221051952377;
Horizon=1; Evaluation of Identity matrix:
  Test set observations = 261237;
  Brier score = 0.035129977759658856;
   

In [28]:
# Now build a single NN using MLP
mlp1 = MLPClassifier(hidden_layer_sizes = (8), activation = 'logistic', max_iter = 1000, random_state = 42,
                   learning_rate_init = 0.001, learning_rate = 'adaptive')
mlp1.fit(X_train, Y_train)

# Get 1 horizon ahead prediction from MLP
pred3 = mlp1.predict_proba(X_test)
eval3 = evaluate(pred3, Y_test)
print_evaluation(eval3, "Horizon=1; Evaluation of MLP:")

# Display transition matrix for basic model without other variables
mlp1_TM = mlp1.predict_proba(np.identity(n_states+1))
np.savetxt("CC_mlp_tm.csv", mlp1_TM, delimiter=",")

Horizon=1; Evaluation of MLP:
  Test set observations = 261237;
  Brier score = 0.02261843360332786;
    by delinquency: [0.06835345 0.03275699 0.01180542 0.00817748 0.00707588 0.00656161
 0.00595183 0.04026481];
  Entropy (mean) = 0.4547432054571421;
    by delinquency: [0.10868299 3.1742829  3.97930428 4.1162434  4.15462826 4.06007405
 4.04412607 1.02872802];
  Roll rate RMSE: 0.002066301588942398;


In [31]:
# propagate predictions forward (horizons)
for horizon in [3,6]:
  print(f"HORIZON={horizon}:")
  X_test, Y_test = encode_dels(CC_b, horizon=horizon)
  print(f"Roll rate (ground truth): {roll_rate(Y_test)}")
  preds1 = predict_from_tm(TM, X_test, horizon)
  eval1 = evaluate(preds1, Y_test)
  print_evaluation(eval1, "Evaluation of Transition matrix:")
  print(f"Roll rate (predictions): {roll_rate(preds1)}")

  # propagate MLP ahead
  P3 = propagate_mlp_mc(mlp1, X_test, horizon)
  eval2 = evaluate(P3, Y_test)
  print_evaluation(eval2, "Evaluation of MLP:")
  print(f"Roll rate (predictions): {roll_rate(P3)}")

HORIZON=3:
Roll rate (ground truth): [0.83503302 0.03362981 0.01198271 0.00844616 0.00728497 0.00674307
 0.0061197  0.09076056]
Evaluation of Transition matrix:
  Test set observations = 245437;
  Brier score = 0.02495229586137255;
    by delinquency: [0.07966166 0.03237444 0.01174299 0.00828959 0.00714295 0.00660143
 0.00599518 0.04781013];
  Entropy (mean) = 0.4990119747705;
    by delinquency: [0.12202761 3.32476852 4.14337054 4.35173965 4.35808088 4.31650804
 4.3692055  1.22637026];
  Roll rate RMSE: 0.006329831381551298;
Roll rate (predictions): [0.83957264 0.03416814 0.0120504  0.00823174 0.00707105 0.00660513
 0.00589984 0.08640107]
Evaluation of MLP:
  Test set observations = 245437;
  Brier score = 0.024893029220180822;
    by delinquency: [0.07969311 0.03237717 0.01174533 0.00828703 0.00714111 0.00659816
 0.00599072 0.0473116 ];
  Entropy (mean) = 0.49970309325347634;
    by delinquency: [0.12686678 3.3202082  4.19976534 4.32566475 4.3312671  4.2495539
 4.25683264 1.20083698]

In [8]:
# also check TM derived from mlp1 (should give the same result as above)
preds3 = predict_from_tm(mlp1_TM, X_test, horizon=horizon)
eval3 = evaluate(preds3, Y_test)
print_evaluation(eval3, f"Horizon={horizon}; Evaluation of Transition matrix derived from MLP:")

Horizon=6; Evaluation of Transition matrix derived from MLP:
  Test set observations = 222012;
  Brier score = 0.030293873037273024;
    by delinquency: [0.10730003 0.03026079 0.01080338 0.00777687 0.00692476 0.0066777
 0.00610448 0.06650298];
  Entropy (mean) = 0.565580460309824;
    by delinquency: [0.15621141 3.35842423 4.34615006 4.52656039 4.5839092  4.50694334
 4.51397023 1.5887132 ];


In [32]:
# Now build MLP with account age
X_train, Y_train = encode_dels(CC_a, include=['acc_age'])
X_test, Y_test = encode_dels(CC_b, include=['acc_age'])

mlp3 = MLPClassifier(hidden_layer_sizes = (16, 8), activation = 'logistic', max_iter = 100, random_state = 42,
                   learning_rate_init = 0.001, learning_rate = 'adaptive')
mlp3.fit(X_train, Y_train)

# Get 1 horizon ahead prediction from MLP
P = mlp3.predict_proba(X_test)
eval3 = evaluate(P, Y_test)
print_evaluation(eval3, "Horizon=1; Evaluation of MLP with acc_age:")

Horizon=1; Evaluation of MLP with acc_age:
  Test set observations = 261237;
  Brier score = 0.021991814493769496;
    by delinquency: [0.06710114 0.03257254 0.01176062 0.00814004 0.00702868 0.006521
 0.00592238 0.03688812];
  Entropy (mean) = 0.4423982572723093;
    by delinquency: [0.10636276 3.14812126 3.8050505  3.91991628 4.02574702 3.99585482
 3.98913336 0.98178994];
  Roll rate RMSE: 0.0017410384796377716;


In [33]:
# propagate MLP ahead
for horizon in [3,6]:
  P3 = propagate_mlp_mc(mlp3, X_test, horizon)
  eval3 = evaluate(P3, Y_test)
  print_evaluation(eval3, f"Horizon={horizon}; Evaluation of MLP with acc_age:")
  print(f"Roll rate (predictions): {roll_rate(P3)}")

Horizon=3; Evaluation of MLP with acc_age:
  Test set observations = 261237;
  Brier score = 0.024141037364547785;
    by delinquency: [0.07758854 0.03327878 0.01191433 0.00821557 0.00710649 0.00658716
 0.00596637 0.04247106];
  Entropy (mean) = 0.4910669601579867;
    by delinquency: [0.13196421 3.32038325 4.08035666 4.17467471 4.28300465 4.23590828
 4.18798381 1.11055749];
  Roll rate RMSE: 0.00914134056412961;
Roll rate (predictions): [0.83346846 0.03280347 0.0121652  0.00907111 0.00716118 0.00666653
 0.00631915 0.0923449 ]
Horizon=6; Evaluation of MLP with acc_age:
  Test set observations = 261237;
  Brier score = 0.02832285623392064;
    by delinquency: [0.0985103  0.0333702  0.01197156 0.00826865 0.00715746 0.00664031
 0.00601521 0.05464916];
  Entropy (mean) = 0.5584878644438266;
    by delinquency: [0.16811281 3.35799121 4.20599884 4.36807148 4.51562019 4.50361406
 4.47073444 1.43124896];
  Roll rate RMSE: 0.020200346244811082;
Roll rate (predictions): [0.82530971 0.03273592 0.

In [34]:
# more variables
features = ['acc_age', 'age', 'maa_log', 'appInternet', 'emp_HO', 'emp_RE', 'emp_SE', 'emp_ST', 'emp_nan']
X_train, Y_train = encode_dels(CC_a, include=features)
X_test, Y_test = encode_dels(CC_b, include=features)
mlp4 = MLPClassifier(hidden_layer_sizes = (32, 16, 8), activation = 'logistic', max_iter = 100, random_state = 42,
                   learning_rate_init = 0.001, learning_rate = 'adaptive')
mlp4.fit(X_train, Y_train)

# Get 1 horizon ahead prediction from MLP
P = mlp4.predict_proba(X_test)
eval3 = evaluate(P, Y_test)
print_evaluation(eval3, "Horizon=1; Evaluation of MLP with acc_age and age:")

Horizon=1; Evaluation of MLP with acc_age and age:
  Test set observations = 261237;
  Brier score = 0.021940729393051025;
    by delinquency: [0.06683584 0.03254711 0.01173831 0.00813725 0.00702754 0.00650475
 0.00591866 0.03681637];
  Entropy (mean) = 0.4380861430956421;
    by delinquency: [0.10660837 3.02357271 3.79752042 3.93038876 3.99089805 3.98919133
 4.02872543 0.98025497];
  Roll rate RMSE: 0.0035513401919319027;


In [35]:
# propagate MLP ahead
for horizon in [3,6]:
  P3 = propagate_mlp_mc(mlp4, X_test, horizon)
  eval3 = evaluate(P3, Y_test)
  print_evaluation(eval3, f"Horizon={horizon}; Evaluation of MLP with all variables:")
  print(f"Roll rate (predictions): {roll_rate(P3)}")

Horizon=3; Evaluation of MLP with all variables:
  Test set observations = 261237;
  Brier score = 0.024300007014977462;
    by delinquency: [0.07874048 0.03322163 0.01190402 0.00821862 0.00710652 0.00658181
 0.00596826 0.04265872];
  Entropy (mean) = 0.4834331948310998;
    by delinquency: [0.12871396 3.17706385 4.05500849 4.19103975 4.27156335 4.2497499
 4.24452887 1.10925112];
  Roll rate RMSE: 0.008999222084675563;
Roll rate (predictions): [0.83219619 0.03702126 0.01202105 0.00874891 0.00712599 0.00632768
 0.00584724 0.09071168]
Horizon=6; Evaluation of MLP with all variables:
  Test set observations = 261237;
  Brier score = 0.028568158765948787;
    by delinquency: [0.10020581 0.03332103 0.01195467 0.00826527 0.00715424 0.00663481
 0.0060151  0.05499433];
  Entropy (mean) = 0.5449290476631513;
    by delinquency: [0.15887989 3.21447058 4.16964647 4.36758671 4.49036248 4.51049964
 4.52075868 1.42484936];
  Roll rate RMSE: 0.019096722951311325;
Roll rate (predictions): [0.82489296 

In [6]:
# more variables & larger MLP
# This taks about 2.5 hours to run on the laptop
features = ['acc_age', 'age', 'maa_log', 'appInternet', 'emp_HO', 'emp_RE', 'emp_SE', 'emp_ST', 'emp_nan']
X_train, Y_train = encode_dels(CC_a, include=features)
X_test, Y_test = encode_dels(CC_b, include=features)
mlp4 = MLPClassifier(hidden_layer_sizes = (128, 128, 32, 8), activation = 'logistic', max_iter = 1000, random_state = 42,
                   learning_rate_init = 0.001, learning_rate = 'adaptive')
mlp4.fit(X_train, Y_train)

# Get 1 horizon ahead prediction from MLP
P = mlp4.predict_proba(X_test)
eval3 = evaluate(P, Y_test)
print_evaluation(eval3, "Horizon=1; Evaluation of MLP with all variables:")

# propagate MLP ahead
for horizon in [3,6]:
  P3 = propagate_mlp_mc(mlp4, X_test, horizon)
  eval3 = evaluate(P3, Y_test)
  print_evaluation(eval3, f"Horizon={horizon}; Evaluation of MLP with all variables:")
  print(f"Roll rate (predictions): {roll_rate(P3)}")

Horizon=1; Evaluation of MLP with all variables:
  Test set observations = 261237;
  Brier score = 0.02261406441483509;
    by delinquency: [0.06898261 0.03276418 0.01182741 0.00816606 0.00706071 0.00654138
 0.00592304 0.03964713];
  Entropy (mean) = 0.45452734346660495;
    by delinquency: [0.10509245 3.19458911 3.9436643  4.17855967 4.13695219 4.14656032
 4.14715886 1.03946825];
  Roll rate RMSE: 0.0025227393620537593;
Horizon=3; Evaluation of MLP with all variables:
  Test set observations = 261237;
  Brier score = 0.02523001039885084;
    by delinquency: [0.08261883 0.03338179 0.01195551 0.0082377  0.00712095 0.00660732
 0.00597093 0.04594705];
  Entropy (mean) = 0.49748249645836884;
    by delinquency: [0.12406833 3.37394069 4.23046439 4.45129209 4.42275531 4.42763189
 4.37507204 1.15240004];
  Roll rate RMSE: 0.008741381455429229;
Roll rate (predictions): [0.83766917 0.03239915 0.0111245  0.00724726 0.00670052 0.00596411
 0.00538806 0.09350722]
Horizon=6; Evaluation of MLP with a

In [ ]:
# roll-rate outcomes from forecasts
# lagged behavioural variables
# lagged UK MEVs